In [5]:
import random
from pathlib import Path
try:
    import megatron
except ImportError:
    !pip install https://github.com/valerio98-lab/Megatron_DeepFake/raw/main/dist/megatron-0.1.0.tar.gz --quiet

import torch
import numpy as np
from megatron.trainer import Trainer
from megatron.configuration import (
    DatasetConfig,
    DataloaderConfig,
    TransformerConfig,
    TrainConfig,
    ExperimentConfig,
)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    experiment = ExperimentConfig(
        dataset=DatasetConfig(
            video_path=Path(r"./drive/MyDrive/Megatron_DeepFake/dataset_processed"),
            num_frames=20,
            random_initial_frame=True,
        ),
        dataloader=DataloaderConfig(
            batch_size=128,
        ),
        transformer=TransformerConfig(
            d_model=384,
            n_heads=6,
            n_layers=4,
            d_ff=2048,
            dropout= 0.1
        ),
        train=TrainConfig(
            learning_rate=0.0001,
            epochs=15,
            tmp_dir= "./drive/MyDrive/Megatron_DeepFake/batches",
            log_dir= "./drive/MyDrive/Megatron_DeepFake/data/main_experiment",
            resume_training=False,
            train_size=0.7,
            val_size=0.2,
        ),
        seed=42,
    )
except ImportError:
    experiment = ExperimentConfig(
        dataset=DatasetConfig(
            video_path=r"G:\My Drive\Megatron_DeepFake\dataset_processed",
            num_frames=5,
            random_initial_frame=False,
            depth_anything_size="Small",
            num_video= 20,
        ),
        dataloader=DataloaderConfig(
            batch_size = 8,
        ),
        transformer=TransformerConfig(
            d_model=600,
            n_heads=6,
            n_layers=4,
            d_ff=2048,
            dropout= 0.3,
            activation = "gelu",
            pooling_type = "max",
        ),
        train=TrainConfig(
            learning_rate=0.001,
            epochs=200,
            tmp_dir= "./../tmp",
            log_dir= "./../data/exp1",
            resume_training=False,
            train_size=0.5,
            val_size=0.3,
            weight_decay =0.1
        ),
        seed=42,
    )
# Set cuda operations deterministic
torch.backends.cudnn.deterministic = True
random.seed(experiment.seed)
torch.manual_seed(experiment.seed)
np.random.seed(experiment.seed)

In [6]:
trainer = Trainer(experiment)
# trainer.optimized_train_and_validate()

In [ ]:
import os 
import torch

path = r"G:\My Drive\Megatron_DeepFake\batches\all\batch_size_128\num_frames_20\depth_anything_size_Small\repvit_model_repvit_m0_9dist_300e_in1k\d_model_384\\"


cnt =0
for elem in os.listdir(path):
    try:
        if "train_rgb" in elem:
            
print(cnt)

In [19]:
%load_ext tensorboard
%tensorboard --logdir experiment.train.log_dir

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
from megatron.model import Megatron

model = Megatron(
    depth_anything_size=experiment.dataset.depth_anything_size,
    repvit_model=experiment.dataloader.repvit_model,
    d_model=experiment.transformer.d_model,
    max_len_pe=experiment.dataset.num_video,
    n_heads=experiment.transformer.n_heads,
    n_layers=experiment.transformer.n_layers,
    d_ff=experiment.transformer.d_ff,
).from_pretrained(Path(experiment.train.log_dir))
model.inference(
    video_path=Path(
        r"H:\My Drive\Megatron_DeepFake\dataset\manipulated_sequences\DeepFakeDetection\raw\videos\01_03__hugging_happy__ISF9SP4G.mp4"
    )
)